# Baseline RAG - landsforsøg 2022 pdf

## Setup

In [1]:
#%pip install llama-index-readers-smart-pdf-loader
#%pip install llama-parse

In [1]:
import os
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb import Settings
from IPython.display import Markdown, display

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core import PromptTemplate, SimpleDirectoryReader

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.chroma import ChromaVectorStore
from openai import OpenAI, AzureOpenAI

from dotenv import load_dotenv

from llmsherpa.readers import LayoutPDFReader
from llama_index.readers.smart_pdf_loader import SmartPDFLoader

#from llama_index.llms.azure_openai import AzureOpenAI
#rom llama_index.embeddings.azure_openai import AzureOpenAIEmbedding


import importlib
import util

#importlib.reload(util.helpers)
from util.helpers import create_and_save_md_files, get_malazan_pages, get_office_pages, get_friends_pages, get_theoffice_pages

In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

In [3]:
# ChromaDB Vector Store
chroma_client = chromadb.PersistentClient(
    path="./data/baseline-rag/chromadb", settings=Settings(allow_reset=True))

openai_client = AzureOpenAI(
    api_key=OPENAI_API_KEY,  
    api_version="2024-05-01-preview", # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-ada-002",
    api_type="azure",
    api_version="2024-05-01-preview"
)

## Load document(s)

### Attempt 1: LayoutPDFReader

In [4]:
llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
pdf = "landsforsøg/planter_landsforsogene_2022.pdf"
doc = pdf_reader.read_pdf(pdf)

In [5]:
len(doc.chunks())

5086

In [6]:
doc.chunks()[0]

Using VectorStoreIndex below yields an Authentication error

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: e611f630********************6e3d. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document

index = VectorStoreIndex([])
for chunk in doc.chunks():
    index.insert(Document(text=chunk.to_context_text(), extra_info={}))
query_engine = index.as_query_engine()

# Let's run one query
response = query_engine.query("list all the tasks that work with bart")
print(response)

In [ ]:
index = VectorStoreIndex.from_documents(doc)
#TypeError: 'Document' object is not iterable

### Attempt 2: SmartPDFLoader
https://llamahub.ai/l/readers/llama-index-readers-smart-pdf-loader?from=

In [4]:
llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_loader = SmartPDFLoader(llmsherpa_api_url=llmsherpa_api_url)
#pdf = "landsforsøg/planter_landsforsogene_2022.pdf"
#documents = pdf_loader.load_data(pdf)

documents = []
for file in os.listdir("landsforsøg"):
    filepath = f"landsforsøg/{file}"
    if "pdf" in filepath:
        print(filepath)
        doc = pdf_loader.load_data(filepath)
        documents.append(doc)

landsforsøg/Husdyrbrugloven.pdf
landsforsøg/planter_landsforsogene_2023.pdf
landsforsøg/planter_landsforsogene_2022.pdf
landsforsøg/FT-73_Klovvaskere_web.pdf


In [ ]:
#authentication error
#index = VectorStoreIndex.from_documents(documents)

In [16]:
from dotenv import load_dotenv

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai_client = AzureOpenAI(
    api_key=OPENAI_API_KEY,  
    api_version="2024-05-01-preview", # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [5]:
document_data = []
for document in documents:
    for idx,doc in enumerate(document):
        embedding = openai_client.embeddings.create(
            input=doc.text, model="text-embedding-ada-002")
        document_data.append({
            "id": f"{doc.id_}-{idx}",
            "text": doc.text,
            "metadata":doc.metadata,
            "embedding": embedding.data[0].embedding
        })

In [12]:
documents = [doc["text"] for doc in document_data]
embeddings = [doc["embedding"] for doc in document_data]
metadatas = [doc["metadata"] for doc in document_data]
ids = [doc["id"] for doc in document_data]

In [13]:
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="landsforsoeg", metadata={"hnsw:space": "cosine"}, embedding_function=openai_ef)

In [14]:
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=ids)

### Retrieval

In [16]:
query = "hvordan kan jeg bedst bekæmpe væselhale?"

In [17]:
result = collection.query(query_texts=[query], n_results=5)
context = result["documents"][0]
#display(Markdown(f"------------\n\n{"\n\n------------\n\n".join(context)}"))

formatted_text = "\n\n------------\n\n".join(context)

# Display the formatted markdown
display(Markdown(f"{formatted_text}"))

MARKFRØ > Rødsvingel > Bekæmpelse af væselhale i rødsvingel
I 2023 er der i samarbejde med DLF videreført en forsøgsserie, som skal belyse mulighederne for bekæmpelse af væselhale i rødsvingel om efteråret.
Væselhale skal bekæmpes så tidligt som muligt efter fremspiring.
Problemet er, at den spirer over en meget lang periode, og derfor er det vanskeligt at dække af for fremspiring.

------------

Svampesygdomme > FOTO: SOFIE HÆSTRUP OLESEN, LANDBONORD
Optimalt tidspunkt for bekæmpelse af væselhale med Boxer og Mateno Duo.

------------

Ukrudt > Bekæmpelse af væselhale om efteråret
Det understreger, at kemisk bekæmpelse af væselhale ikke kan stå alene, men at bestanden skal bringes ned på et niveau, hvor de tilbageværende væselhale efter kemisk bekæmpelse er få og uden betydning for fortsat opformering og spredning.

------------

Svampesygdomme > FOTO: SOFIE HÆSTRUP OLESEN, LANDBONORD
TABEL 18.
Bekæmpelse af væselhale i vinterhvede om efter- året.
(E16) Vinterhvede Stadie Væselhale Kemi og udbring- ning, kr.
pr.
ha Oktober November Antal planter pr.
m2 Antal planter pr.
m2 Procent effekt 2021-22, 3 forsøg

------------

Ukrudt > Bekæmpelse af væselhale om efteråret
Der er udført tre forsøg i vinterhvede med bekæmpelse af væselhale med forskellige strategier med Boxer, Ma- teno Duo og Atlantis OD i henholdsvis stadie 10-11 og stadie 12.
Behandlingerne ses i tabel 18.
Forsøgene er udført på arealer med en meget stor be- stand af væselhale, i gennemsnit ca.
600 planter pr.
m2 ved optælling i oktober.
Den tidlige sprøjtning i stadie 10-11 er udført fra 6 til 14 dage efter såning, som i gen- nemsnit har været midt i september.
Anden sprøjtning i Forsøgsled 2 og 6 viser, at der er opnået samme effekt- niveau af 1,5 l Boxer pr.
ha og 0,7 l Mateno Duo pr.
ha.
I forsøgsled 3 til 5 er forskellige blandingsforhold mel- lem Boxer og Mateno Duo afprøvet, hvilket samlet er en højere indsats.
Effekten er dermed også lidt bedre.
Re- sultatet viser også i disse forsøgsled, at der har været et ligeværdigt bidrag fra begge midler.

## Generation

In [18]:
query = "hvordan kan jeg bedst bekæmpe væselhale?"

prompt = PromptTemplate("""You are a helpful assistant that answers questions about landsforsøgene using provided context. You must provide your answer in the Danish language.

Question: {query}

Context: 

-----------------------------------
{context}

-----------------------------------

""")
message = prompt.format(query=query, context="\n\n".join(context))
display(Markdown(f"{message}"))

You are a helpful assistant that answers questions about landsforsøgene using provided context. You must provide your answer in the Danish language.

Question: hvordan kan jeg bedst bekæmpe væselhale?

Context: 

-----------------------------------
MARKFRØ > Rødsvingel > Bekæmpelse af væselhale i rødsvingel
I 2023 er der i samarbejde med DLF videreført en forsøgsserie, som skal belyse mulighederne for bekæmpelse af væselhale i rødsvingel om efteråret.
Væselhale skal bekæmpes så tidligt som muligt efter fremspiring.
Problemet er, at den spirer over en meget lang periode, og derfor er det vanskeligt at dække af for fremspiring.

Svampesygdomme > FOTO: SOFIE HÆSTRUP OLESEN, LANDBONORD
Optimalt tidspunkt for bekæmpelse af væselhale med Boxer og Mateno Duo.

Ukrudt > Bekæmpelse af væselhale om efteråret
Det understreger, at kemisk bekæmpelse af væselhale ikke kan stå alene, men at bestanden skal bringes ned på et niveau, hvor de tilbageværende væselhale efter kemisk bekæmpelse er få og uden betydning for fortsat opformering og spredning.

Svampesygdomme > FOTO: SOFIE HÆSTRUP OLESEN, LANDBONORD
TABEL 18.
Bekæmpelse af væselhale i vinterhvede om efter- året.
(E16) Vinterhvede Stadie Væselhale Kemi og udbring- ning, kr.
pr.
ha Oktober November Antal planter pr.
m2 Antal planter pr.
m2 Procent effekt 2021-22, 3 forsøg

Ukrudt > Bekæmpelse af væselhale om efteråret
Der er udført tre forsøg i vinterhvede med bekæmpelse af væselhale med forskellige strategier med Boxer, Ma- teno Duo og Atlantis OD i henholdsvis stadie 10-11 og stadie 12.
Behandlingerne ses i tabel 18.
Forsøgene er udført på arealer med en meget stor be- stand af væselhale, i gennemsnit ca.
600 planter pr.
m2 ved optælling i oktober.
Den tidlige sprøjtning i stadie 10-11 er udført fra 6 til 14 dage efter såning, som i gen- nemsnit har været midt i september.
Anden sprøjtning i Forsøgsled 2 og 6 viser, at der er opnået samme effekt- niveau af 1,5 l Boxer pr.
ha og 0,7 l Mateno Duo pr.
ha.
I forsøgsled 3 til 5 er forskellige blandingsforhold mel- lem Boxer og Mateno Duo afprøvet, hvilket samlet er en højere indsats.
Effekten er dermed også lidt bedre.
Re- sultatet viser også i disse forsøgsled, at der har været et ligeværdigt bidrag fra begge midler.

-----------------------------------



In [19]:
query = "hvordan kan jeg bedst bekæmpe væselhale?"

stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": query}],
    model="gpt4",
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

Væselhale er en invasiv ukrudtstype, der kan være vanskelig at kontrollere og udrydde. Her er nogle metoder til bekæmpelse af væselhale:

1. Mekanisk bekæmpelse: Hvis du kun observerer væselhale på små områder, kan man fysisk fjerne ukrudtet ved hånden. Det er vigtigt at sørge for at fjerne hele planten inklusiv roden for at forhindre, at den vokser igen.

2. Anvendelse af herbicider: Vælg midler specifikt designet til at bekæmpe græs-lignende ukrudt, og følg vejledningen på produktet nøje for at opnå de bedste resultater. Der er dog blevet observeret resistens overfor visse herbicider hos væselhale, så det kan være nødvendigt at kombinere forskellige metoder.

3. Bekæmp væselhale ved at ændre jordens forhold eller afgrøderotation. Væxler du mellem forskellige afgrøder, kan det forstyrre væselhalens livscyklus og reducere frøproduktionen.

4. Biologisk bekæmpelse: Frigivelse af naturlige fjender (såsom visse insekter eller pattedyr) kan også bidrage til at kontrollere ukrudtsudbredelsen.

5. Forbedre græssets sundhed: Ved at passe godt på græsplænen kan du gøre det sværere for væselhalen at vokse. Dette inkluderer regelmæssig vanding, gødning, og korrekt klipning.

6. Hvis du har en stor infestation, kan det være nødvendigt at søge professionel hjælp. En professionel ukrudtsbekæmpelsestjeneste har adgang til stærkere kemikalier og mere effektive metoder.

Husk altid at håndtere ukrudtsmidler forsigtigt, da de kan være skadelige for både mennesker og miljø, hvis de ikke bruges korrekt.

## Normal RAG example with llamaindex

In [21]:
import chromadb
import os

from dotenv import load_dotenv
from chromadb import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

load_dotenv()

# ChromaDB Vector Store
chroma_client = chromadb.PersistentClient(
    path="./landsforsøg/data/baseline-rag/chromadb", settings=Settings(allow_reset=True))
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="landsforsoeg", metadata={"hnsw:space": "cosine"})
vector_store = ChromaVectorStore(chroma_collection=collection)


llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="gpt4",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embedding = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Define the ingestion pipeline to add documents to vector store
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=20),
        embedding,
    ],
    vector_store=vector_store,
)

# Create index with the vector store and using the embedding model
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embedding)

In [22]:
# Fetch documents
documents = SimpleDirectoryReader('./landsforsøg').load_data()

# Run pipeline
pipeline.run(documents=documents)

print("Done")

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 662 0 (offset 0)


Done


In [44]:
from llama_index.core import PromptTemplate
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer

    
qa_prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the content of documents and provides detailed expert advice. 
    You must provide your answer in the Danish language.
    If the answer contains multiple steps or points, provide the answer in a bullet format.
    Below the answer, the source of the answer should be provided including file_name and page number.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {query_str}
    Answer: 
    """,
)


class RAGQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: OpenAI
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        context_str = "\n\n".join([n.node.get_content(metadata_mode="all") for n in nodes])
        #context = qa_prompt.format(
        #    context_str=context_str, query_str=query_str)
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )
                    
        return str(response) + "\n\n-------------------------\n\nKontekst:\n\n" + context_str


synthesizer = get_response_synthesizer(response_mode="compact")
query_engine = RAGQueryEngine(
    retriever=index.as_retriever(),
    response_synthesizer=synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
)

In [55]:
from IPython.display import Markdown, display

#query = "hvem udgiver landsforsøgene?"
#query = "hvordan kan jeg bedst bekæmpe væselhale?"
query = "hvordan vælger jeg den bedste vårbygsort?"
#query = "hvad er reglerne for afstande ved etablering af husdyranlæg?"
#query = "Beskriv MT-Klovvask"
response = query_engine.query(query)
display(Markdown(f"{response}"))

For at vælge den bedste vårbygsort, skal du overveje følgende faktorer:

- Vælg en sort, der har givet et stort og stabilt udbytte i flere års forsøg.
- Vælg en sort, der har lav modtagelighed over for sygdomme, i prioriteret rækkefølge: meldug, bygrust, skoldplet og bygbladplet.
- Vælg en sort, der har resistens mod havrecystenematoder.
- Vælg en sort, der har en god stråstivhed, så der ikke er behov for vækstregulering.
- Vælg en sort, der har en svag tendens til nedknækning af aks og strå.
- Hvis du dyrker vårbyg til malt, skal du altid vælge en maltbygsort, der er accepteret af handelspartneren.

Kilde: planter_landsforsogene_2023.pdf, side 77 og planter_landsforsogene_2022.pdf, side 89.

-------------------------

Kontekst:

page_label: 77
file_name: planter_landsforsogene_2023.pdf
file_path: /Users/peerchristensen/Desktop/Projects/advanced-rag-examples/landsforsøg/planter_landsforsogene_2023.pdf
file_type: application/pdf
file_size: 47095474
creation_date: 2024-06-08
last_modified_date: 2024-06-08

Der var mange grønskud i vårbyg i 2023. Her er det en prøve fra 
et forsøg med et vandindhold på over 30 procent.FOTO: LEIF HAGELSKJÆR, SEGES INNOVATION
STRATEGI
Vælg en vårbygsort, der:
 >har givet et stort og stabilt udbytte i flere års for -
søg
 >har lav modtagelighed over for sygdommene (i 
prioriteret rækkefølge):
 – meldug
 – bygrust
 – skoldplet og bygbladplet
 >har resistens mod havrecystenematoder
 >har en god stråstivhed, så der ikke er behov for 
vækstregulering
 >har en svag tendens til nedknækning af aks og 
strå.
Ved dyrkning af vårbyg til malt bør der altid vælges 
en maltbygsort, der er accepteret af handelspart -
neren.

page_label: 89
file_name: planter_landsforsogene_2022.pdf
file_path: /Users/peerchristensen/Desktop/Projects/advanced-rag-examples/landsforsøg/planter_landsforsogene_2022.pdf
file_type: application/pdf
file_size: 53911931
creation_date: 2024-06-07
last_modified_date: 2024-06-07

17. maj. Forsøget er sået d. 21. marts.FOTO: LEIF HAGELSKJÆR, SEGES INNOVATION
STRATEGI
Vælg en vårbygsort, der:
 >har givet et stort og stabilt udbytte i flere års for -
søg
 >har lav modtagelighed over for sygdommene (i 
prioriteret rækkefølge):
– meldug
– bygrust
– skoldplet og bygbladplet
 >har resistens mod havrecystenematoder
 >har en god stråstivhed, så der ikke er behov for 
vækstregulering
 >har en svag tendens til nedknækning af aks og 
strå.
Ved dyrkning af vårbyg til malt bør der altid vælges 
en maltbygsort, der er accepteret af handelspart -
neren.